<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S03_Data_Structures_1/03_Lecture_Example2_SC_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S3 - Python Data Structures I - SC examples

Supply chain examples include:
1. Forecasting methods
 - Naive (random walk and seasonal)
 - Moving Average
 - Simple exponential smoothing
2. Forecasting error measures: mean squared error (MSE), mean absolute percentage error (MAPE)
3. Optimal parameter search for forecasting methods using Python

*NOTE:* the examples are used to show the applications of data structures covered in Session 3. It is also possible to use a more advanced data structure (in particular DataFrame) for this as well (which will be covered in Session 4).

### Example 1: Naive Forecasting Method
In the simple naive forecasting method, we simply use the observed demand of the previous period $t$ as the forecast for period $t+1$. Formally, we have

> $$F_{t+1}=D_{t}$$
> Where:
> - $F_{t+1}$: forecast for the period $t+1$
> - $D_t$: demand for period $t$

This method is referred to as "random walk" forecasts which work surprisingly well in predicting some financial time series (e.g., stock prices).

Given the historical sales below of the past 12 months
- Historical sales = `[125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]`

We can write a function to calculate the naive forecast of a given period $t$.

In [1]:
# first we prepare the list for historical demands (sales)
sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]


In [2]:
# defining the naive forecast function
def naiveForecast(historical_sales, t):
  """
  Return the predicted demand for the next period using the naive forecast
  parameters:
      historical_sales: (list) real sales in the previous periods
      t: (int number) period to forecast
  return:
      forecast for period t
  """
  return historical_sales[t-1]


In [3]:
t = 12 # this is period 13 as the first index starts at zero. We cannot go beyond this as we have only 12 elements in the list (indexed 0 -> 11)
print("Naive forecast for t = ", t, " is ", "{:.2f}".format(naiveForecast(sales, t))) #the use of "{:.2f}".format(x) is to print out exactly 2 digits

Naive forecast for t =  12  is  126.00


Alternatively, in the **seasonal** naive forecast, we can look back to the same period in the previous seasonality cycle and take that value as the forecast (e.g., using sales in Jan 2021 as the forecast for Jan 2022). We can extend the previous function for this case.

In [4]:
# defining the naive forecast function
def seasonalNaiveForecast(historical_sales, t, s):
  """
    Return the predicted demand for the next period using the seasonal naive forecast
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        s: (int number) length of seasonality cycle
    return:
        forecast for period t
  """
  return historical_sales[t-s]

In [5]:
t = 12 # period to forecast
s = 6 # seasonality cycle
print("Seasonal Naive forecast for t = ", t, "and s =", s," is ", "{:.2f}".format(seasonalNaiveForecast(sales, t,s)))

Seasonal Naive forecast for t =  12 and s = 6  is  128.00


### Example 2: Moving Average Forecasting Method

The moving average is a time series method which uses data on past demand to predict future demand. As other time series methods, moving average is based on the assumption that history repeats itself and, therefore, they ignore changes in the environment than can affect future demand.

> **Brief description of the model:** this method involves computing average demand for the $k$ most recent periods and using it as the forecast of demand. This method can be represented using the following equation:
>
> $$F_{t+1}=\frac{D_t+D_{t-1}+D_{t-2}+...+D_{(t-k)+1}}{k}$$
> Where:
> - $k$: number of observations used in the calculation
>
> For example, if k = 3, the forecast of period $21$ can be calculated by: $F_{21}=\frac{D_{20}+D_{19}+D_{18}}{3}$.
> The choice of the number of periods ($k$) to consider in order to make the demand forecast is important:
> - If $k$ is small, the forecast will react quickly to real changes (i.e., variations in demand that are not random), but they will also be influenced to a greater extent by random variations;
>- if $k$ is large, forecast will be less affected by random variations in demand, but also slower to react to real changes.

Given a list with historical sales and the value of $k$, we want to predict the sales volume for the next time period.
- Predict future sales using $k=2$
- Predict future sales using $k=3$



In [6]:
# defining the moving average function
def movingAvg(historical_sales, t, k):
    """
    Return the predicted demand for the next period
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        k: (int number) parameter of the moving avg method
    return:
        forecast for period t
    """
    return sum(historical_sales[t-k:t]) / k

t = 12 # period to forecast

print("Moving average forecast (k = 2) for t = ", t, " is ", "{:.2f}".format(movingAvg(sales, t, 2)))
print("Moving average forecast (k = 3) for t = ", t, " is ", "{:.2f}".format(movingAvg(sales, t, 3)))

Moving average forecast (k = 2) for t =  12  is  129.00
Moving average forecast (k = 3) for t =  12  is  130.67


### Example 3: Exponential Smoothing Forecasting Method

Using an initial demand forecast, a smoothing factor $0 \leq \alpha \leq 1$, we can calculate the forecast using the smoothing parameter as follows.

$$ F_{t+1}=\alpha D_t + (1-\alpha) F_t $$

Where:    
- $\alpha$ = choice of smoothing parameter.

Consequently, we first need to calculate $F_t$ which is basically equal to 
$ F_{t}=\alpha D_{t-1} + (1-\alpha) F_{t-1}$.


Thus, we will need to calculate the first forecast (forecast period 2 from period 1) and *iteratively* calculate the forecast from period 2 until $t + 1$.  

Given a list with historical sales and the value of $\alpha$, we can create a function which uses the simple exponential smoothing method to calculate the forecast demand for a given period.
- Predict future sales using $\alpha= 0.2$
- Predict future sales using $\alpha= 0.5$

In [7]:
# defining the exponential smooting function
def exponentialSmoothing(historical_sales, t, alpha):
    """
    Return the predicted demand for the next period
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        alpha: smoothing parameter
    return:
        forecast for period t
    """
    exp_forecast = [] # start with an empty list

    # We can use list.append() to add an element to the list
    exp_forecast.append(historical_sales[0]) # assume the initial forecast (index 0) = actual demand in the same period
    # calculate forecast for period 1 until t iteratively
    for i in range(1, t+1):
      exp_forecast.append(alpha*historical_sales[i-1]+(1-alpha)*exp_forecast[i-1])

    # print("exp_forecast list: ", exp_forecast) # you can outcomment this to show the produced list
    return  exp_forecast[t]

t = 12 # period to forecast
print("Exponential smoothing forecast (alpha = 0.2) for t = ", t, " is ", "{:.2f}".format(exponentialSmoothing(sales, t, 0.2)))
print("Exponential smoothing forecast (alpha = 0.5) for t = ", t, " is ", "{:.2f}".format(exponentialSmoothing(sales, t, 0.5)))

Exponential smoothing forecast (alpha = 0.2) for t =  12  is  131.54
Exponential smoothing forecast (alpha = 0.5) for t =  12  is  129.37


### Example 4: Forecasting measures

**Mean Squared Error (MSE)**

Different forecasting methods can provide a different forecast quality. In order to estimate the quality of a forecast, some measures are used in practice, including the mean squared error (MSE).

MSE measures the quadratic deviation of forecast and actual data according to the following equation.

$$ MSE = \frac{1}{T}\sum_{t=1}^{T}(D_t-F_t)^2$$

Given two inputs: (i) a list of demand forecast and (ii) a list of demand realizations, create a function which returns the MSE. First, we define the function.

In [8]:
# define a function which computes MSE
def MSE(forecast, real_demand):
    """
    Compute the MSE 
    parameters:
        forecast: (list) demand forecast for a given planning horizon
        real_demand: (list) real demand over a given planning horizon
        Attention: the lists of real_demand and forecast must be of the same size
    return:
        MSE: (number) mean squared error
    """
    sum_mse = 0
    n_periods = len(forecast) # get the number of periods from the list
    for t in range(n_periods):
        sum_mse += (real_demand[t] - forecast[t]) ** 2
    return sum_mse/n_periods

Next, we call the function to know the MSE of our forecast.

In [9]:
# Consider a list of real sales and predictions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MSE of our predictions is: ", "{:.2f}".format(MSE(predictions, real_sales)))

The MSE of our predictions is:  62.83


Alternatively, we can also use *list comprehension* instead of *for* loop

In [10]:
# define a function which computes MSE using list comprehension
def MSE(forecast, real_demand):
    n_periods = len(forecast)
    mse_t = [(real_demand[t] - forecast[t]) ** 2 for t in range(n_periods)] # MSE for each time period t 
    return sum(mse_t)/n_periods # we use the function sum() to calculate the sum of mse_t

# Consider a list of real sales and predictions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MSE of our predictions is: ", "{:.2f}".format(MSE(predictions, real_sales)))

The MSE of our predictions is:  62.83



**Mean absolute percentage errors (MAPE)**

Another commonly used forecasting measure is the MAPE which can be calculated as follows.

$$ MAPE = \frac{1}{T}\sum_{t=1}^{T}\left|\frac{D_t-F_t}{D_t}\right|$$

which can be implemented as follows:

In [11]:
# define a function which computes MAPE using list comprehension
def MAPE(forecast, real_demand):
    """
    Compute the MAPE 
    parameters:
        forecast: (list) demand forecast for a given planning horizon
        real_demand: (list) real demand over a given planning horizon
        Attention: the lists of real_demand and forecast must be of the same size
    return:
        Mean absolute percentage errors (MAPE)
    """
    n_periods = len(forecast)
    mape_t = [abs(real_demand[t] - forecast[t])/real_demand[t] for t in range(n_periods)] 
    return sum(mape_t)/n_periods # we use the function sum() to calculate the sum of mape_t

# Consider a list of real sales and predictions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MAPE of our predictions is: ", "{:.2f}".format(MAPE(predictions, real_sales)*100),"%")

The MAPE of our predictions is:  4.05 %


## Example 5: Forecasting and measuring performances

We are putting them all together here. We will try different forecasting models with different configurations and measure their performances. The results will be stored in a (nested) dictionary.

We will use the methods defined earlier and find the best configuration for each method when predicting the prices of United States Oil Fund (USO) which tracks the price of West Texas Intermediate Light Sweet Crude in 2019. The forecasts and errors will be calculated for the second half of 2019 (i.e., elements with index 6-11) to be measured against the actual values).



In [12]:
uso_2019 = [90.80, 95.60, 100.00, 106.32, 88.80, 96.32, 96.31, 91.68, 90.72, 90.40, 92.96, 102.48]

First, one-step Naive method as a baseline model

In [13]:
uso_fcst_naive = [naiveForecast(uso_2019, t) for t in range(6, 12)]
print(uso_fcst_naive)

uso_fcst_naive_mse = MSE(uso_fcst_naive, uso_2019[6:])
print("MSE = ", uso_fcst_naive_mse)

uso_fcst_naive_mape = MAPE(uso_fcst_naive, uso_2019[6:])
print("MAPE = ", uso_fcst_naive_mape)

# We can then keep the results as a dictionary 
fcst_results = {} # first create an empty dictionary
fcst_names = ['Naive', 'SeasonNaive', 'MovingAvg', 'Exp.Smooth'] # list of forecasting methods
print("add results to the dictionary for the method: ", fcst_names[0])
fcst_results[fcst_names[0]] = {'Forecast':uso_fcst_naive, 'MSE':uso_fcst_naive_mse, 'MAPE':uso_fcst_naive_mape}
print(fcst_results)

[96.32, 96.31, 91.68, 90.72, 90.4, 92.96]
MSE =  19.940833333333348
MAPE =  0.030860385227782578
add results to the dictionary for the method:  Naive
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}}


Then, we apply the seasonal Naive forecast and search for the best seasonnality length $s$ between 2, 4 and 6 based on MAPE.

In [14]:
best_s = 0 # initialize the initial value of best s
best_mape = 1.0 # initialize the initial value of MAPE (at the maximum 100%)

for s in range(2,7,2): # here we can use range(start, finish, stepsize) to create an increment of 2, or, alternatively, use the list [2,4,6]
  print("s = ", s)
  uso_fcst_seasonnaive = [seasonalNaiveForecast(uso_2019, t, s) for t in range(6, 12)]
  uso_fcst_seasonnaive_mape = MAPE(uso_fcst_seasonnaive, uso_2019[6:])
  print("MAPE :", uso_fcst_seasonnaive_mape)
  if uso_fcst_seasonnaive_mape < best_mape: # keep track of the best s
    best_s = s # set the new best_s
    best_mape = uso_fcst_seasonnaive_mape # set the new best_mape

print("best seasonality length s = ", best_s)
# we compute again the corresponding forecasts and results based on best_s
uso_fcst_seasonnaive = [seasonalNaiveForecast(uso_2019, t, best_s) for t in range(6, 12)]
uso_fcst_seasonnaive_mse = MSE(uso_fcst_seasonnaive, uso_2019[6:])
uso_fcst_seasonnaive_mape = MAPE(uso_fcst_seasonnaive, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[1])
fcst_results[fcst_names[1]] = {'Forecast':uso_fcst_seasonnaive, 'MSE':uso_fcst_seasonnaive_mse, 'MAPE':uso_fcst_seasonnaive_mape}
print(fcst_results)

s =  2
MAPE : 0.057723114537770515
s =  4
MAPE : 0.07101230185854347
s =  6
MAPE : 0.08053786497565112
best seasonality length s =  2
add results to the dictionary for the method:  SeasonNaive
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}}


Now we apply the moving average forecast and search for the best lookback period $k$ between 2 to 5 based on MAPE.

In [15]:
best_k = 0 # initialize the initial value of best k
best_mape = 1.0 # initialize the initial value of MAPE (at the maximum 100%)

for k in range(2,6):
  print("k = ", k)
  uso_fcst_movingavg = [movingAvg(uso_2019, t, k) for t in range(6, 12)]
  uso_fcst_movingavg_mape = MAPE(uso_fcst_movingavg, uso_2019[6:])
  print("MAPE :", uso_fcst_movingavg_mape)
  if uso_fcst_movingavg_mape < best_mape: # keep track of the best parameter
    best_k = k # set the new best_k
    best_mape = uso_fcst_movingavg_mape # set the new best_mape

print("best lookback length k = ", best_k)
# we compute again the corresponding forecasts and results based on best_s
uso_fcst_movingavg = [movingAvg(uso_2019, t, best_k) for t in range(6, 12)]
uso_fcst_movingavg_mse = MSE(uso_fcst_movingavg, uso_2019[6:])
uso_fcst_movingavg_mape = MAPE(uso_fcst_movingavg, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[2])
fcst_results[fcst_names[2]] = {'Forecast':uso_fcst_movingavg, 'MSE':uso_fcst_movingavg_mse, 'MAPE':uso_fcst_movingavg_mape}
print(fcst_results)

k =  2
MAPE : 0.044274444653136145
k =  3
MAPE : 0.03909421269515928
k =  4
MAPE : 0.042306982792044284
k =  5
MAPE : 0.04302070201943891
best lookback length k =  3
add results to the dictionary for the method:  MovingAvg
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}, 'MovingAvg': {'Forecast': [97.14666666666666, 93.81, 94.77, 92.90333333333335, 90.93333333333334, 91.36], 'MSE': 25.944644444444453, 'MAPE': 0.03909421269515928}}


Lastly, we apply the exponential smoothing forecast and search for the best $\alpha$ between 0.1 to 0.9 with a stepsize of 0.1 based on MAPE.

In [16]:
best_alpha = 0 # initialize the initial value of best alpha
best_mape = 1.0 # initialize the initial value of MAPE (at the maximum 100%)

for alpha in [0.1*i for i in range(1,10)]: # create a list of [0.1,...,0.8] using list comprehension
  print("alpha = ", "{:.1f}".format(alpha))
  uso_fcst_expsmooth = [exponentialSmoothing(uso_2019, t, alpha) for t in range(6, 12)]
  uso_fcst_expsmooth_mape = MAPE(uso_fcst_expsmooth, uso_2019[6:])
  print("MAPE :", uso_fcst_expsmooth_mape)
  if uso_fcst_expsmooth_mape < best_mape: # keep track of the best parameter
    best_alpha = alpha # set the new best parameter
    best_mape = uso_fcst_expsmooth_mape # set the new best_mape

print("best alpha = ", best_alpha)
# we compute again the corresponding forecasts and results based on best_s
uso_fcst_expsmooth = [exponentialSmoothing(uso_2019, t, best_alpha) for t in range(6, 12)]
uso_fcst_expsmooth_mse = MSE(uso_fcst_expsmooth, uso_2019[6:])
uso_fcst_expsmooth_mape = MAPE(uso_fcst_expsmooth, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[3])
fcst_results[fcst_names[3]] = {'Forecast':uso_fcst_expsmooth, 'MSE':uso_fcst_expsmooth_mse, 'MAPE':uso_fcst_expsmooth_mape}
print(fcst_results)

alpha =  0.1
MAPE : 0.03455040544430794
alpha =  0.2
MAPE : 0.03740061515515078
alpha =  0.3
MAPE : 0.038183022048744
alpha =  0.4
MAPE : 0.038291632918183605
alpha =  0.5
MAPE : 0.03781851315454915
alpha =  0.6
MAPE : 0.03707021866224728
alpha =  0.7
MAPE : 0.03611894256920158
alpha =  0.8
MAPE : 0.034869814598145234
alpha =  0.9
MAPE : 0.03315885402784039
best alpha =  0.9
add results to the dictionary for the method:  Exp.Smooth
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}, 'MovingAvg': {'Forecast': [97.14666666666666, 93.81, 94.77, 92.90333333333335, 90.93333333333334, 91.36], 'MSE': 25.944644444444453, 'MAPE': 0.03909421269515928}, 'Exp.Smooth': {'Forecast': [95.736392, 96.2526392, 92.13726392000001, 90.861726392, 90.44617263920001, 92.70861726392], 'MSE': 20.876522040985435, 'MA

We can then print out all the results in multiple lines for MSE and MAPE. (Un)surprisingly, the naive method turns out to be the best for this time-series, followed by the exponential smoothing with $\alpha = 0.9$:

In [17]:
for method in fcst_names:
  print("Method:", method, ", MSE = ", "{:.2f}".format(fcst_results[method]['MSE']), #if the code is too long, you can also split at ,
        ", MAPE = ", "{:.2f}".format(fcst_results[method]['MAPE']*100),"%")

Method: Naive , MSE =  19.94 , MAPE =  3.09 %
Method: SeasonNaive , MSE =  43.63 , MAPE =  5.77 %
Method: MovingAvg , MSE =  25.94 , MAPE =  3.91 %
Method: Exp.Smooth , MSE =  20.88 , MAPE =  3.32 %


**Supplement:** we can also use a similar process to "combine" two (or even more) forecasts using the weight $0 \leq w \leq 1$ which can be optimized. For example, we can combine the best SeasonNaive and MovingAvg, i.e., $w\times SeasonNaive + (1-w)\times MovingAvg$ as follows:

In [20]:
best_w = 0 # initialize the initial value of best w
best_mape = 1.0 # initialize the initial value of MAPE (at the maximum 100%)

for w in [0.1*i for i in range(10)]: # create a list of [0.0,...,1.0] using list comprehension
  print("aggregation weight w = ", "{:.1f}".format(w))
  uso_fcst_combined = [w* fcst_results['SeasonNaive']['Forecast'][t] + 
                       (1-w)*fcst_results['MovingAvg']['Forecast'][t] for t in range(6)]
  # print(uso_fcst_combined)
  uso_fcst_combined_mape = MAPE(uso_fcst_combined, uso_2019[6:])

  print("MAPE :", uso_fcst_combined_mape)
  if uso_fcst_combined_mape < best_mape: # keep track of the best parameter
    best_w = w # set the new best parameter
    best_mape = uso_fcst_combined_mape # set the new best_mape

aggregation weight w =  0.0
MAPE : 0.03909421269515928
aggregation weight w =  0.1
MAPE : 0.03835785738743272
aggregation weight w =  0.2
MAPE : 0.04050339965582033
aggregation weight w =  0.3
MAPE : 0.04265586401606416
aggregation weight w =  0.4
MAPE : 0.04480832837630788
aggregation weight w =  0.5
MAPE : 0.04696079273655168
aggregation weight w =  0.6
MAPE : 0.049113257096795426
aggregation weight w =  0.7
MAPE : 0.05126572145703925
aggregation weight w =  0.8
MAPE : 0.053418185817282936
aggregation weight w =  0.9
MAPE : 0.05557065017752679


You can see that the best MAPE (3.8%) with $w = 0.1$ is actually better than the best of either method (either $w=0.0$ or $w=1.0$) in this case.